In [1]:
import sys
import os

notebook_dir = os.path.dirname(os.path.abspath('__file__'))
src_dir = os.path.join(notebook_dir, '../../')
sys.path.append(src_dir)

In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from src.metrics.metrics import compute_metrics, default_regression_metrics
from src.preprocessing.preprocessing import Preprocessor
from src.io.input import load_artifacts
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso, Ridge, LinearRegression
from sklearn.preprocessing import StandardScaler
from src.pipelines.training_pipelines import StandardTrainerPipeline
from src.train.training import train_multiple_models
from src.models.default import get_default_regression_models
from src.train.cross_validation import cross_validate
from src.metrics.metrics import summarize_cv_metrics

import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('../../data/training_data.csv')
print(df.shape)

(800, 21)


In [4]:
df.head(2)

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,target
0,432.475954,289.373016,481.315600,358.755566,802.659004,176.761177,72.648102,720.969179,36.327684,83.768878,...,4.385848,516.789458,19.624422,13.162440,42.351948,35.920392,20.755984,13.814300,384.497136,14.364922
1,517.596250,330.448341,585.920055,22.684031,169.813240,335.601640,284.451476,748.101047,73.701438,358.147215,...,5.563334,2.960064,20.721878,17.740184,1.726915,167.576065,75.492679,2.480979,303.710869,19.984801


### 1. Exploratory Data Analysis

In [5]:
#report = ProfileReport(df)
#report.to_file('report.html')

In [6]:
df_train, df_val = train_test_split(df, test_size=0.2)
print(df_train.shape, df_val.shape)

(640, 21) (160, 21)


### 2. Preprocessing

In [7]:
# Preprocessing params
target_column = 'target'
numerical_features = list(df_train.drop(columns=[target_column]).columns)
categorical_features = []
preprocessing_graph = ['individual_num', 'combinated_num']
artifacts = {}
# Fit preprocessors in training mode
train_pipeline = Preprocessor(df_train,
                              'train',
                              numerical_features=numerical_features,
                              categorical_features=categorical_features,
                              target_column=target_column,
                              graph_preprocess=preprocessing_graph,
                              artifacts=artifacts
                             )
df_train_preproc = train_pipeline.preprocess()

# Load generated artifacts
artifacts = load_artifacts('../../artifacts/')
# Apply same preprocess logic to validation set
validation_pipeline = Preprocessor(df_val,
                                   'val',
                                   numerical_features=numerical_features,
                                   categorical_features=categorical_features,
                                   target_column=target_column,
                                   graph_preprocess=preprocessing_graph,
                                   artifacts=artifacts
                                  )
df_val_preproc = validation_pipeline.preprocess()

### 3. Metrics choice

In [8]:
metrics_config = default_regression_metrics()

### 4. Model training

In [14]:
list_models = [XGBRegressor(), LGBMRegressor(verbose=-1),
               RandomForestRegressor(), LinearRegression(),
               Ridge(), Lasso()#, create_neural_network(df_train_preproc.shape[1]-1),
               ]
list_model_names = ['xgboost', 'lightgbm', 'random_forest',
                    'linear_regression', 'ridge', 'lasso'#, 'dnn'
                   ]

metrics_df, fitted_models = train_multiple_models(df_train_preproc,
                                                  df_val_preproc,
                                                  list_models,
                                                  list_model_names,
                                                  target_column,
                                                  metrics_config
                                                 )

In [15]:
metrics_df

,model,dataset,mae,mse,rmse,mape,r2
0,xgboost,train,2.020502e-04,6.616232e-08,2.572204e-04,1.876692e-05,1.000000
0,xgboost,validation,1.913595e+00,5.441831e+00,2.332773e+00,1.676546e-01,0.793309
0,lightgbm,train,4.522762e-02,1.520974e-02,1.233278e-01,1.232418e-02,0.999407
0,lightgbm,validation,1.574255e+00,4.173158e+00,2.042831e+00,1.449852e-01,0.841496
0,random_forest,train,7.781453e-01,9.677586e-01,9.837472e-01,8.180614e-02,0.962256
0,random_forest,validation,1.997142e+00,6.256958e+00,2.501391e+00,1.829239e-01,0.762349
0,linear_regression,train,1.809481e-14,5.563086e-28,2.358619e-14,1.889896e-15,1.000000
0,linear_regression,validation,8.820487e+00,3.005985e+03,5.482686e+01,5.928812e-01,-113.173023
0,ridge,train,4.208623e-04,3.016522e-07,5.492288e-04,3.471786e-05,1.000000
0,ridge,validation,8.818468e+00,3.004199e+03,5.481057e+01,5.927545e-01,-113.105171


### 5. Complete training pipeline

In [11]:
target_column = 'target'
numerical_features = list(df.drop(columns=[target_column]).columns)
categorical_features = []
preprocessing_graph = ['individual_num']#, 'combinated_num']
artifacts = {}
test_size = 0.2

list_models, list_model_names = get_default_regression_models()
metrics_config = default_regression_metrics()

training_pipeline = StandardTrainerPipeline(df.copy(),
                                            numerical_features,
                                            categorical_features,
                                            target_column,
                                            preprocessing_graph,
                                            artifacts.copy(),
                                            list_models.copy(),
                                            list_model_names,
                                            metrics_config,
                                            test_size
                                           )
training_pipeline.run()

In [12]:
training_pipeline.metrics_df

,model,dataset,mae,mse,rmse,mape,r2
0,xgboost,train,0.001036,0.000002,0.001466,0.000079,1.000000
0,xgboost,validation,2.034968,6.360839,2.522070,0.195321,0.749091
0,lightgbm,train,0.114356,0.036674,0.191504,0.015787,0.998571
0,lightgbm,validation,1.660830,4.114566,2.028439,0.156451,0.837697
0,random_forest,train,0.839668,1.141729,1.068517,0.089564,0.955517
0,random_forest,validation,2.151080,6.787356,2.605255,0.210413,0.732267
0,linear_regression,train,1.241687,2.711592,1.646691,0.110095,0.894353
0,linear_regression,validation,2.066803,7.832096,2.798588,0.180543,0.691056
0,ridge,train,1.351632,3.093961,1.758966,0.128260,0.879456
0,ridge,validation,1.841866,6.258797,2.501759,0.156608,0.753116


### 6. Cross-validation

In [16]:
target_column = 'target'
numerical_features = list(df.drop(columns=[target_column]).columns)
categorical_features = []
preprocessing_graph = ['individual_num']#, 'combinated_num']
artifacts = {}
test_size = 0.2
num_folds = 5

list_models, list_model_names = get_default_regression_models()
metrics_config = default_regression_metrics()

metrics_df, all_models = cross_validate(df.copy(),
                                        list_models,
                                        list_model_names,
                                        target_column,
                                        metrics_config,
                                        num_folds,
                                        artifacts,
                                        numerical_features,
                                        categorical_features,
                                        preprocessing_graph,
                                        test_size
                                       )

Fold 0
Fold 1
Fold 2
Fold 3
Fold 4


In [19]:
summarize_cv_metrics(metrics_df, 'mape')

,dataset,model,mae_cv,mse_cv,rmse_cv,mape_cv,r2_cv
,,,,,,,
5,train,xgboost,0.0007 +/- 0.0002,0.0 +/- 0.0,0.001 +/- 0.0003,0.0001 +/- 0.0,1.0 +/- 0.0
1,train,lightgbm,0.1184 +/- 0.0022,0.0362 +/- 0.0018,0.1901 +/- 0.0048,0.0137 +/- 0.0027,0.9986 +/- 0.0001
3,train,random_forest,0.8177 +/- 0.021,1.0849 +/- 0.0445,1.0414 +/- 0.0214,0.0794 +/- 0.0058,0.958 +/- 0.0025
2,train,linear_regression,1.2475 +/- 0.0381,2.6353 +/- 0.1154,1.623 +/- 0.0355,0.1044 +/- 0.0052,0.898 +/- 0.0045
4,train,ridge,1.3451 +/- 0.0351,2.9698 +/- 0.1202,1.723 +/- 0.035,0.1182 +/- 0.0072,0.8851 +/- 0.0045
0,train,lasso,1.5444 +/- 0.0272,3.9225 +/- 0.1556,1.9802 +/- 0.0395,0.137 +/- 0.0056,0.8482 +/- 0.0055
6,validation,lasso,1.7002 +/- 0.1184,4.8368 +/- 0.7539,2.1937 +/- 0.1749,0.1563 +/- 0.0206,0.8119 +/- 0.0323
10,validation,ridge,1.8606 +/- 0.1969,7.4262 +/- 3.2361,2.6752 +/- 0.5807,0.1768 +/- 0.0361,0.7131 +/- 0.1205
8,validation,linear_regression,2.1468 +/- 0.2304,11.332 +/- 5.624,3.2959 +/- 0.7656,0.1974 +/- 0.028,0.5647 +/- 0.2017


In [23]:
all_models[0]['lightgbm']

LGBMRegressor(verbose=-1)

In [54]:
from lightgbm import LGBMRegressor
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from functools import partial
from src.train.training import fit_model


def objective_score(search_space):
  """
  Objective function to optimize, it returns the value of the loss function
  for some specific model iteration on some hyperparameters combination.
  Args:
    search_space - Grid of hyperparameters where the search will be performed
  Returns:
    loss - Value of the loss function for the given hyperparameters
    status - Status object indicating that the iteration went smoothly
  """
  # TODO: Make objects come from args
  model = LGBMRegressor(**search_space)
  model_name = 'lgbm'
    
  fitted_model, model_metrics = fit_model(model,
                                        model_name,
                                        df_train_preproc,
                                        df_val_preproc,
                                        'target',
                                        metrics_config
                                      )

  test_mse = float(model_metrics[model_metrics['dataset']=='validation']['mse'])
  test_mae = float(model_metrics[model_metrics['dataset']=='validation']['mae'])
  test_mape = float(model_metrics[model_metrics['dataset']=='validation']['mape'])
  print(f'mse: {test_mse} - mae: {test_mae} - mape: {test_mape}')

  return {'loss': test_mse, 'status': STATUS_OK }


In [62]:
top_model = all_models[0]['lightgbm']
search_space = top_model.get_params().copy()

search_space['max_depth'] = hp.uniformint('max_depth', 1, 500, q=1)
search_space['num_leaves'] = hp.uniformint('num_leaves', 30, 1000, q=1)
search_space['boosting_type'] = hp.choice('boosting_type', ['gbdt', 'dart'])
search_space['colsample_bytree'] = hp.uniform('colsample_bytree', 0.6, 1.0)
search_space['learning_rate'] = 0.1
search_space['reg_alpha'] = hp.uniform('reg_alpha', 0.0, 1000.0)
search_space['reg_lambda'] = hp.uniform('reg_lambda', 0.0, 1000.0)

In [63]:
df_train_preproc = training_pipeline.df_train_preproc
df_val_preproc = training_pipeline.df_val_preproc

In [ ]:
from functools import partial

In [64]:
trials = Trials()
best_hyperparams = fmin(fn=objective_score,
                        space=search_space,
                        algo=tpe.suggest,
                        max_evals=10,
                        trials=trials)

{'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.9130093507977841, 'importance_type': 'split', 'learning_rate': 0.1, 'max_depth': 149, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 100, 'n_jobs': None, 'num_leaves': 943, 'objective': None, 'random_state': None, 'reg_alpha': 714.3153302034979, 'reg_lambda': 633.817418103553, 'subsample': 1.0, 'subsample_for_bin': 200000, 'subsample_freq': 0, 'verbose': -1}
mse: 25.576808012707094 - mae: 4.126630602953107 - mape: 0.4604473678422954       
{'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.8452912334512592, 'importance_type': 'split', 'learning_rate': 0.1, 'max_depth': 25, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 100, 'n_jobs': None, 'num_leaves': 340, 'objective': None, 'random_state': None, 'reg_alpha': 501.42719147970286, 'reg_lambda': 845.8957423869093, 'subsample': 1.0, 'subsample_for_bin': 200000, 's